In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-2/dataset/corpus_hf.txt"
testDataPath = "s3://corpus-2/dataset/corpus_5_new.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
split_info = {
    "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}


def dataset_split(info):
    if info:
        [num, pi] = info
        train_data = [[] for i in range(num)]
        test_data = [[] for i in range(num)]
        
        with open(trainDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                train_data[index].append([content,label])
                
        with open(testDataPath, "r", encoding='utf-8') as fp:
            for line in fp.readlines():
                word = line.split()
                info = word[0].split(":")
                index = int(info[pi]) - 1
                label = int(info[0])
                content = word[1:]
                test_data[index].append([content,label])

        for i in range(num):
            # np.random.shuffle(train_data[i])
            train_data[i], test_data[i] = shuffle(train_data[i], test_data[i])
            train_data[i] = np.asarray(train_data[i])
            test_data[i] = np.asarray(test_data[i])

        train_data, test_data = shuffle(train_data, test_data)
        return train_data, test_data
    
    
    train_data = []
    test_data = []
    
    with open(trainDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            train_data.append([content,label])
            
    with open(testDataPath, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            word = line.split()
            label = int(word[0].split(":")[0])
            content = word[1:]
            test_data.append([content,label])
            
    train_data, test_data = shuffle(train_data, test_data)
    return np.asarray(train_data), np.asarray(test_data)


def mergeData(data_x, data_y):
    merge_x = data_x[0]
    merge_y = data_y[0]
    for i in range(1,len(data_x)):
        merge_x = np.r_[merge_x,data_x[i]]
        merge_y = np.r_[merge_y,data_y[i]]
        
    return merge_x, merge_y


def train_split_data(model, train_data, test_data, split_type):
    
    print(split_type)
    
    test_acc = []
    fold_id = 0
    
    if split_type != "random":
        tx = []
        ty = []
        for ti in train_data:
            x_train, y_train = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            tx.append(x_train)
            ty.append(y_train)

        tx = np.asarray(tx)
        ty = np.asarray(ty)

        te_x = []
        te_y = []
        for ti in test_data:
            x_test, y_test = process_file(ti[:,0], ti[:,1], word_to_id, num_classes, seq_length)
            te_x.append(x_test)
            te_y.append(y_test)

        te_x = np.asarray(te_x)
        te_y = np.asarray(te_y)
        
        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            train_x, train_y = mergeData(tx[train_i],ty[train_i])
            test_x, test_y = mergeData(te_x[test_i],te_y[test_i])
            test_acc.append(model_train(model, train_x, train_y, test_x, test_y, categories))
        
    else:
        tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, num_classes, seq_length)
        te_x, te_y = process_file(test_data[:,0], test_data[:,1], word_to_id, num_classes, seq_length)
        # print(len(tx),len(tx[0]),len(tx[1]))

        for train_i, test_i in kf.split(tx):
            fold_id += 1
            print("Fold: ", fold_id)
            test_acc.append(model_train(model,tx[train_i], ty[train_i],te_x[test_i], te_y[test_i], categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(labels[i] - 1)  # label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [5]:
categories = ['Retrieve Value', 'Filter', 'Compute Derived Value', 'Find Extremum', 'Sort', 
                  'Determine Range', 'Characterize Distribution', 'Find Anomalies', 'Cluster', 'Correlate']
num_classes = len(categories)

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)
 
seq_length = 41  # seq_length = 37  TREC

Loading GloVe!
Completed!
400000 100 400000


In [6]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, model, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {model.inputX: x_batch1, model.inputY: y_batch1, model.dropoutKeepProb: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(model, x_train, y_train, x_val, y_val, categories):
    
    # save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    require_improvement = 400  # 如果超过1000轮未提升，提前结束训练
    print_per_batch = 100
    flag = False

    for epoch in range(num_epochs):  # 20
        start_time = time.time()
        
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {model.inputX: x_batch, model.inputY: y_batch, model.dropoutKeepProb: dropout_keep_prob}
            session.run(model.trainOp, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[model.dropoutKeepProb] = 1.0
                loss_train, acc_train = session.run([model.loss, model.acc], feed_dict=feed_dict)
                loss_val, acc_val = evaluate(session, model, x_val, y_val, model.loss, model.acc, 64)
                if acc_val > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_val
                    last_improved = total_batch
                    # saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''
                
                duration = time.time() - start_time
                output = 'Iter: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
                print(output.format(total_batch, loss_train, acc_train, loss_val, acc_val, duration, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("%s:%s  %s" % (total_batch, last_improved, "No optimization for a long time, auto-stopping..."))
                
                test_data_len = len(x_val)
                test_num_batch = int((test_data_len - 1) / batch_size) + 1

                y_test_cls = np.argmax(y_val, 1)  # 获得类别
                y_test_pred_cls = np.zeros(shape=len(x_val), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

                for i in range(test_num_batch):  # 逐批次处理
                    start_id = i * batch_size
                    end_id = min((i + 1) * batch_size, test_data_len)
                    feed_dict = {
                        model.inputX: x_val[start_id:end_id],
                        model.dropoutKeepProb: 1.0
                    }
                    y_test_pred_cls[start_id:end_id] = session.run(model.y_pred_cls, feed_dict=feed_dict)

                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                # 评估
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                # 混淆矩阵
                print("Confusion Matrix...")
                cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                print(cm)
                
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return accuracy_score

In [7]:
# 构建adversarailLSTM模型
class AdversarailLSTM(object):

    def __init__(self, wordEmbedding):
        # 定义输入
        self.inputX = tf.placeholder(tf.int32, [None, seq_length], name='inputX')
        self.inputY = tf.placeholder(tf.int32, [None, num_classes], name='inputY')

        self.dropoutKeepProb = tf.placeholder(tf.float64, name='keep_prob')

        # 词嵌入层
        with tf.name_scope("wordEmbedding"):
            wordEmbedding = tf.Variable(initial_value=wordEmbedding)
            self.embeddedWords = tf.nn.embedding_lookup(wordEmbedding, self.inputX)

        # 计算softmax交叉熵损失
        with tf.name_scope("loss"):
            with tf.variable_scope("Bi-LSTM", reuse=None):
                self.predictions = self._Bi_LSTMAttention(self.embeddedWords)
                # self.y_pred_cls = tf.cast(tf.greater_equal(self.predictions, 0.5), tf.float32, name="binaryPreds")
                self.y_pred_cls = tf.argmax(tf.nn.softmax(self.predictions),1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）
                # losses = tf.nn.sigmoid_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.predictions, labels=self.inputY)
                loss = tf.reduce_mean(losses)

        
        with tf.name_scope("perturloss"):
            with tf.variable_scope("Bi-LSTM", reuse=True):
                perturWordEmbedding = self._addPerturbation(self.embeddedWords, loss)
                print("perturbSize:{}".format(perturWordEmbedding))
                perturPredictions = self._Bi_LSTMAttention(perturWordEmbedding)
                # perturLosses = tf.nn.sigmoid_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLosses = tf.nn.softmax_cross_entropy_with_logits(logits=perturPredictions, labels=self.inputY)
                perturLoss = tf.reduce_mean(perturLosses)

        self.loss = loss + perturLoss
        
        globalStep = tf.Variable(0, name="globalStep", trainable=False)
        # 定义优化函数，传入学习速率参数
        optimizer = tf.train.AdamOptimizer(learning_rate)
        # 计算梯度,得到梯度和变量
        gradsAndVars = optimizer.compute_gradients(self.loss)
        # 将梯度应用到变量下，生成训练器
        self.trainOp = optimizer.apply_gradients(gradsAndVars, global_step=globalStep)

        # 准确率
        correct_pred = tf.equal(tf.argmax(self.inputY, 1), self.y_pred_cls)
        self.acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        # self.loss = loss
        
        
    def _Bi_LSTMAttention(self, embeddedWords):
        # 定义两层双向LSTM的模型结构
        with tf.name_scope("Bi-LSTM"):
            fwHiddenLayers = []
            bwHiddenLayers = []
            for idx, hiddenSize in enumerate(hiddenSizes):
                with tf.name_scope("Bi-LSTM" + str(idx)):
                    # 定义前向网络结构
                    lstmFwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                    # 定义反向网络结构
                    lstmBwCell = tf.nn.rnn_cell.DropoutWrapper(
                        tf.nn.rnn_cell.LSTMCell(num_units=hiddenSize, state_is_tuple=True),
                        output_keep_prob=self.dropoutKeepProb)

                fwHiddenLayers.append(lstmFwCell)
                bwHiddenLayers.append(lstmBwCell)

            # 实现多层的LSTM结构， state_is_tuple=True，则状态会以元祖的形式组合(h, c)，否则列向拼接
            fwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=fwHiddenLayers, state_is_tuple=True)
            bwMultiLstm = tf.nn.rnn_cell.MultiRNNCell(cells=bwHiddenLayers, state_is_tuple=True)
            # 采用动态rnn，可以动态地输入序列的长度，若没有输入，则取序列的全长
            # outputs是一个元组(output_fw, output_bw), 其中两个元素的维度都是[batch_size, max_time, hidden_size], fw和bw的hiddensize一样
            # self.current_state是最终的状态，二元组(state_fw, state_bw), state_fw=[batch_size, s], s是一个元组(h, c)
            outputs, self.current_state = tf.nn.bidirectional_dynamic_rnn(fwMultiLstm, bwMultiLstm,
                                                                          self.embeddedWords, dtype=tf.float64,
                                                                          scope="bi-lstm" + str(idx))

        # 在bi-lstm+attention论文中，将前向和后向的输出相加
        with tf.name_scope("Attention"):
            H = outputs[0] + outputs[1]

            # 得到attention的输出
            output = self.attention(H)
            outputSize = hiddenSizes[-1]
            print("outputSize:{}".format(outputSize))

        # 全连接层的输出
        with tf.name_scope("output"):
            outputW = tf.get_variable(
                "outputW", dtype=tf.float64,
                shape=[outputSize, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())

            outputB = tf.Variable(tf.constant(0.1, dtype=tf.float64, shape=[num_classes]), name="outputB")

            predictions = tf.nn.xw_plus_b(output, outputW, outputB, name="predictions")

            return predictions

    def attention(self, H):
        """
        利用Attention机制得到句子的向量表示
        """
        # 获得最后一层lstm神经元的数量
        hiddenSize = hiddenSizes[-1]

        # 初始化一个权重向量，是可训练的参数
        W = tf.Variable(tf.random_normal([hiddenSize], stddev=0.1, dtype=tf.float64))

        # 对bi-lstm的输出用激活函数做非线性转换
        M = tf.tanh(H)

        # 对W和M做矩阵运算，W=[batch_size, time_step, hidden_size], 计算前做维度转换成[batch_size * time_step, hidden_size]
        # newM = [batch_size, time_step, 1], 每一个时间步的输出由向量转换成一个数字
        newM = tf.matmul(tf.reshape(M, [-1, hiddenSize]), tf.reshape(W, [-1, 1]))

        # 对newM做维度转换成[batch_size, time_step]
        restoreM = tf.reshape(newM, [-1, seq_length])

        # 用softmax做归一化处理[batch_size, time_step]
        self.alpha = tf.nn.softmax(restoreM)

        # 利用求得的alpha的值对H进行加权求和，用矩阵运算直接操作
        r = tf.matmul(tf.transpose(H, [0, 2, 1]), tf.reshape(self.alpha, [-1, seq_length, 1]))

        # 将三维压缩成二维sequeezeR = [batch_size, hissen_size]
        sequeezeR = tf.squeeze(r)

        sentenceRepren = tf.tanh(sequeezeR)

        # 对attention的输出可以做dropout处理
        output = tf.nn.dropout(sentenceRepren, self.dropoutKeepProb)

        return output

    def _normalize(self, wordEmbedding, weights):
        """
        对word embedding 结合权重做标准化处理
        """
        mean = tf.matmul(weights, wordEmbedding)
        powWordEmbedding = tf.pow(wordEmbedding - mean, 2.)

        var = tf.matmul(weights, powWordEmbedding)
        stddev = tf.sqrt(1e-6 + var)

        return (wordEmbedding - mean) / stddev

    def _addPerturbation(self, embedded, loss):
        """
        添加波动到word embedding
        """
        grad, = tf.gradients(
            loss,
            embedded,
            aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)
        grad = tf.stop_gradient(grad)
        perturb = self._scaleL2(grad, epsilon)
        # print("perturbSize:{}".format(embedded+perturb))
        return embedded + perturb

    def _scaleL2(self, x, norm_length):
        # shape(x) = [batch, num_step, d]
        # divide x by max(abs(x)) for a numerically stable L2 norm
        # 2norm(x) = a * 2norm(x/a)
        # scale over the full sequence, dim(1, 2)
        alpha = tf.reduce_max(tf.abs(x), (1, 2), keep_dims=True) + 1e-12
        l2_norm = alpha * tf.sqrt(tf.reduce_sum(tf.pow(x / alpha, 2), (1, 2), keep_dims=True) + 1e-6)
        x_unit = x / l2_norm
        return norm_length * x_unit

In [8]:
hiddenSizes = [128]  # 定义LSTM的隐藏层（一层，128个神经元）
epsilon = 5

num_filters = 256
kernel_size = 5
hidden_dim = 128
learning_rate = 1e-3
dropout_keep_prob = 0.5

num_epochs = 50
batch_size = 64
print_per_batch = 30  # 每多少轮输出一次结果

lstm = AdversarailLSTM(embedding)

outputSize:128
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


perturbSize:Tensor("perturloss/Bi-LSTM/add_2:0", shape=(?, 41, 100), dtype=float64)
outputSize:128


In [9]:
kf = KFold(n_splits=10)
split_info = {
    # "random": False,
    # "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}
test_acc_split = []
for split_type,info in split_info.items():
    train_data,test_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, test_data, split_type))

bundle
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.821, Train Acc: 56.25%, Val Loss:  3.326, Val Acc: 46.08%, Time: 16.88s *
Epoch: 2
Iter: 200, Train Loss:  1.663, Train Acc: 76.56%, Val Loss:  2.663, Val Acc: 58.87%, Time: 1.40s *
Iter: 300, Train Loss:  1.456, Train Acc: 71.88%, Val Loss:  2.424, Val Acc: 61.19%, Time: 13.95s *
Epoch: 3
Iter: 400, Train Loss: 0.7608, Train Acc: 89.06%, Val Loss:  2.165, Val Acc: 66.06%, Time: 1.63s *
Iter: 500, Train Loss:  1.042, Train Acc: 82.81%, Val Loss:  2.175, Val Acc: 68.10%, Time: 14.18s *
Epoch: 4
Iter: 600, Train Loss: 0.5504, Train Acc: 93.75%, Val Loss:  2.243, Val Acc: 69.62%, Time: 1.87s *
Iter: 700, Train Loss: 0.3786, Train Acc: 93.75%, Val Loss:   2.49, Val Acc: 66.86%, Time: 14.42s 
Epoch: 5
Iter: 800, Train Loss: 0.2374, Train Acc: 96.88%, Val Loss:   2.22, Val Acc: 69.26%, Time: 2.09s 
Iter: 900, Train Loss: 0.8383, Train Acc: 93.75%, Val Loss:  2.385, Val Acc: 70.28%, Time: 14.63s *
Epoch: 6
Iter: 100

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.963, Train Acc: 56.25%, Val Loss:  3.346, Val Acc: 44.54%, Time: 13.63s *
Epoch: 2
Iter: 200, Train Loss:  1.802, Train Acc: 73.44%, Val Loss:  2.688, Val Acc: 55.81%, Time: 1.58s *
Iter: 300, Train Loss:  1.899, Train Acc: 64.06%, Val Loss:  2.445, Val Acc: 61.93%, Time: 14.16s *
Epoch: 3
Iter: 400, Train Loss: 0.7006, Train Acc: 90.62%, Val Loss:  2.342, Val Acc: 65.90%, Time: 1.91s *
Iter: 500, Train Loss: 0.5905, Train Acc: 92.19%, Val Loss:  2.354, Val Acc: 66.60%, Time: 14.56s *
Epoch: 4
Iter: 600, Train Loss: 0.3504, Train Acc: 93.75%, Val Loss:  2.439, Val Acc: 66.39%, Time: 2.26s 
Iter: 700, Train Loss: 0.5337, Train Acc: 93.75%, Val Loss:  2.543, Val Acc: 67.57%, Time: 14.89s *
Epoch: 5
Iter: 800, Train Loss:  0.314, Train Acc: 95.31%, Val Loss:  2.523, Val Acc: 69.24%, Time: 2.59s *
Iter: 900, Train Loss: 0.6006, Train Acc: 93.75%, Val Loss:  2.575, Val Acc: 68.27%, Time: 15.18s 
Epoch: 6
Iter: 1000, Train Loss: 0

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  3.026, Train Acc: 57.81%, Val Loss:   3.45, Val Acc: 41.67%, Time: 13.50s *
Epoch: 2
Iter: 200, Train Loss:  1.616, Train Acc: 78.12%, Val Loss:  2.476, Val Acc: 60.92%, Time: 1.41s *
Iter: 300, Train Loss: 0.9923, Train Acc: 87.50%, Val Loss:  2.281, Val Acc: 64.94%, Time: 13.99s *
Epoch: 3
Iter: 400, Train Loss: 0.7665, Train Acc: 87.50%, Val Loss:  2.162, Val Acc: 67.03%, Time: 1.64s *
Iter: 500, Train Loss:  1.238, Train Acc: 78.12%, Val Loss:  2.412, Val Acc: 65.01%, Time: 14.22s 
Epoch: 4
Iter: 600, Train Loss: 0.5247, Train Acc: 93.75%, Val Loss:  2.374, Val Acc: 67.60%, Time: 1.87s *
Iter: 700, Train Loss: 0.9574, Train Acc: 82.81%, Val Loss:  2.598, Val Acc: 65.59%, Time: 14.40s 
Epoch: 5
Iter: 800, Train Loss: 0.3413, Train Acc: 96.88%, Val Loss:  2.516, Val Acc: 66.31%, Time: 2.10s 
Iter: 900, Train Loss: 0.2888, Train Acc: 96.88%, Val Loss:  2.684, Val Acc: 65.73%, Time: 14.64s 
Epoch: 6
Iter: 1000, Train Loss: 0.2

Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.63      0.60      0.61       151
                   Filter       0.64      0.73      0.68       147
    Compute Derived Value       0.64      0.68      0.66       142
            Find Extremum       0.81      0.80      0.81       163
                     Sort       0.96      0.79      0.87       168
          Determine Range       0.69      0.84      0.76       126
Characterize Distribution       0.87      0.75      0.81       165
           Find Anomalies       0.76      0.84      0.80       140
                  Cluster       0.81      0.85      0.83        78
                Correlate       0.85      0.79      0.82       121

                micro avg       0.76      0.76      0.76      1401
                macro avg       0.77      0.77      0.76      1401
             weighted avg       0.77      0.76      0.76      1401

Confusion Matrix...
[[ 90

Iter: 1500, Train Loss: 0.1566, Train Acc: 98.44%, Val Loss:  2.376, Val Acc: 71.39%, Time: 16.34s 
Epoch: 9
Iter: 1600, Train Loss: 0.01279, Train Acc: 100.00%, Val Loss:    2.7, Val Acc: 72.06%, Time: 4.03s 
1601:1200  No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.80      0.55      0.65       150
                   Filter       0.58      0.55      0.57       137
    Compute Derived Value       0.54      0.72      0.61       185
            Find Extremum       0.84      0.79      0.81       192
                     Sort       0.76      0.78      0.77       157
          Determine Range       0.70      0.76      0.73       163
Characterize Distribution       0.85      0.76      0.80        98
           Find Anomalies       0.72      0.82      0.77       120
                  Cluster       0.95      0.74      0.83       129
                Correlat

Epoch: 6
Iter: 1100, Train Loss: 0.2989, Train Acc: 96.88%, Val Loss:  2.439, Val Acc: 67.26%, Time: 7.49s 
Iter: 1200, Train Loss: 0.5312, Train Acc: 95.31%, Val Loss:  2.615, Val Acc: 65.62%, Time: 19.58s 
Epoch: 7
Iter: 1300, Train Loss: 0.4035, Train Acc: 96.88%, Val Loss:  2.526, Val Acc: 66.98%, Time: 6.58s 
Iter: 1400, Train Loss: 0.06921, Train Acc: 100.00%, Val Loss:  2.778, Val Acc: 66.71%, Time: 18.59s 
1401:1000  No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.40      0.49      0.44        79
                   Filter       0.56      0.44      0.49       119
    Compute Derived Value       0.77      0.55      0.64       111
            Find Extremum       0.70      0.86      0.77        74
                     Sort       0.60      1.00      0.75        29
          Determine Range       0.66      0.82      0.73        73
Characterize Dist

Iter: 1200, Train Loss: 0.09278, Train Acc: 100.00%, Val Loss:  2.608, Val Acc: 68.32%, Time: 21.58s 
1201:800  No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.54      0.54      0.54        91
                   Filter       0.58      0.58      0.58        88
    Compute Derived Value       0.66      0.62      0.64       102
            Find Extremum       0.65      0.69      0.67        95
                     Sort       0.81      0.83      0.82        88
          Determine Range       0.76      0.64      0.70        95
Characterize Distribution       0.71      0.93      0.80        94
           Find Anomalies       0.61      0.64      0.63        90
                  Cluster       0.76      0.72      0.74       112
                Correlate       0.73      0.62      0.67        92

                micro avg       0.68      0.68      0.68       94

In [11]:
kf = KFold(n_splits=10)
split_info = {
    # "random": False,
    # "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}
test_acc_split = []
for split_type,info in split_info.items():
    train_data,test_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, test_data, split_type))

bundle
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  3.302, Train Acc: 32.81%, Val Loss:  3.443, Val Acc: 40.69%, Time: 17.16s *
Epoch: 2
Iter: 200, Train Loss:    1.6, Train Acc: 73.44%, Val Loss:  2.404, Val Acc: 60.24%, Time: 1.72s *
Iter: 300, Train Loss:  1.362, Train Acc: 76.56%, Val Loss:  2.065, Val Acc: 67.15%, Time: 17.61s *
Epoch: 3
Iter: 400, Train Loss: 0.6613, Train Acc: 89.06%, Val Loss:  2.011, Val Acc: 69.16%, Time: 2.01s *
Iter: 500, Train Loss: 0.8318, Train Acc: 82.81%, Val Loss:   2.01, Val Acc: 71.03%, Time: 17.97s *
Epoch: 4
Iter: 600, Train Loss: 0.6473, Train Acc: 89.06%, Val Loss:   1.96, Val Acc: 72.90%, Time: 2.27s *
Iter: 700, Train Loss: 0.6764, Train Acc: 85.94%, Val Loss:   1.98, Val Acc: 73.62%, Time: 18.18s *
Epoch: 5
Iter: 800, Train Loss: 0.4024, Train Acc: 93.75%, Val Loss:  2.069, Val Acc: 73.04%, Time: 2.58s 
Iter: 900, Train Loss: 0.4899, Train Acc: 92.19%, Val Loss:  2.173, Val Acc: 73.26%, Time: 18.58s 
Epoch: 6
Iter: 100

InvalidArgumentError: In[0] is not a matrix
	 [[Node: perturloss/Bi-LSTM/output/predictions/MatMul = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](perturloss/Bi-LSTM/Attention/dropout/mul, Bi-LSTM/outputW/read)]]
	 [[Node: Adam/update/_46 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3036_Adam/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'perturloss/Bi-LSTM/output/predictions/MatMul', defined at:
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/p

In [10]:
kf = KFold(n_splits=10)
split_info = {
    # "random": False,
    "expert": [20, 4],
    "bundle": [920, 1],
    "table": [37, 3]
}
test_acc_split = []
for split_type,info in split_info.items():
    train_data,test_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, test_data, split_type))

expert
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.775, Train Acc: 65.62%, Val Loss:  3.457, Val Acc: 42.28%, Time: 16.90s *
Iter: 200, Train Loss:  1.661, Train Acc: 80.00%, Val Loss:  2.836, Val Acc: 54.66%, Time: 32.69s *
Epoch: 2
Iter: 300, Train Loss:  1.506, Train Acc: 79.69%, Val Loss:  2.706, Val Acc: 61.74%, Time: 15.86s *
Iter: 400, Train Loss:  1.065, Train Acc: 85.45%, Val Loss:  2.616, Val Acc: 62.94%, Time: 31.60s *
Epoch: 3
Iter: 500, Train Loss:  0.544, Train Acc: 93.75%, Val Loss:  2.638, Val Acc: 62.06%, Time: 15.81s 
Iter: 600, Train Loss:  1.217, Train Acc: 89.09%, Val Loss:   2.65, Val Acc: 63.75%, Time: 31.55s *
Epoch: 4
Iter: 700, Train Loss: 0.6386, Train Acc: 92.19%, Val Loss:  2.684, Val Acc: 65.19%, Time: 15.65s *
Iter: 800, Train Loss: 0.5776, Train Acc: 90.91%, Val Loss:  2.863, Val Acc: 64.71%, Time: 31.40s 
Epoch: 5
Iter: 900, Train Loss: 0.9272, Train Acc: 92.19%, Val Loss:  2.832, Val Acc: 64.15%, Time: 15.81s 
Iter: 1000, Tra

Iter: 700, Train Loss: 0.3203, Train Acc: 95.31%, Val Loss:  1.699, Val Acc: 75.23%, Time: 17.44s 
Epoch: 5
Iter: 800, Train Loss: 0.3017, Train Acc: 96.88%, Val Loss:  1.653, Val Acc: 77.58%, Time: 1.96s *
Iter: 900, Train Loss: 0.1636, Train Acc: 100.00%, Val Loss:   1.75, Val Acc: 76.44%, Time: 17.71s 
Epoch: 6
Iter: 1000, Train Loss: 0.2408, Train Acc: 95.31%, Val Loss:   1.76, Val Acc: 77.13%, Time: 2.06s 
Iter: 1100, Train Loss: 0.2898, Train Acc: 95.31%, Val Loss:  1.736, Val Acc: 75.53%, Time: 17.93s 
Epoch: 7
Iter: 1200, Train Loss: 0.1073, Train Acc: 100.00%, Val Loss:   1.65, Val Acc: 78.27%, Time: 2.24s *
Iter: 1300, Train Loss: 0.3604, Train Acc: 92.19%, Val Loss:  1.804, Val Acc: 78.65%, Time: 18.06s *
Epoch: 8
Iter: 1400, Train Loss: 0.05368, Train Acc: 100.00%, Val Loss:  2.015, Val Acc: 75.30%, Time: 2.36s 
Iter: 1500, Train Loss: 0.1219, Train Acc: 98.44%, Val Loss:  1.983, Val Acc: 76.82%, Time: 18.11s 
Epoch: 9
Iter: 1600, Train Loss: 0.0948, Train Acc: 98.44%, Val 

InvalidArgumentError: In[0] is not a matrix
	 [[Node: perturloss/Bi-LSTM/output/predictions/MatMul = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](perturloss/Bi-LSTM/Attention/dropout/mul, Bi-LSTM/outputW/read)]]
	 [[Node: Adam/update/_46 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3036_Adam/update", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'perturloss/Bi-LSTM/output/predictions/MatMul', defined at:
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/p

In [9]:
kf = KFold(n_splits=10)
test_acc_split = []
for split_type,info in split_info.items():
    train_data,test_data = dataset_split(info)
    test_acc_split.append(train_split_data(lstm, train_data, test_data, split_type))

random
Fold:  1
Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.743, Train Acc: 54.69%, Val Loss:  3.002, Val Acc: 52.49%, Time: 18.40s *
Epoch: 2
Iter: 200, Train Loss:  2.108, Train Acc: 65.62%, Val Loss:  1.883, Val Acc: 70.37%, Time: 1.72s *
Iter: 300, Train Loss:  1.015, Train Acc: 87.50%, Val Loss:  1.525, Val Acc: 75.57%, Time: 17.80s *
Epoch: 3
Iter: 400, Train Loss: 0.9277, Train Acc: 85.94%, Val Loss:  1.245, Val Acc: 82.26%, Time: 2.04s *
Iter: 500, Train Loss: 0.6279, Train Acc: 90.62%, Val Loss:  1.102, Val Acc: 83.97%, Time: 17.91s *
Epoch: 4
Iter: 600, Train Loss: 0.7389, Train Acc: 92.19%, Val Loss:  1.029, Val Acc: 84.47%, Time: 2.30s *
Iter: 700, Train Loss: 0.2953, Train Acc: 95.31%, Val Loss: 0.8694, Val Acc: 87.25%, Time: 18.37s *
Epoch: 5
Iter: 800, Train Loss: 0.2237, Train Acc: 96.88%, Val Loss:  0.823, Val Acc: 87.68%, Time: 2.62s *
Iter: 900, Train Loss: 0.2388, Train Acc: 96.88%, Val Loss: 0.8893, Val Acc: 85.75%, Time: 18.65s 
Epoch: 6
Iter: 10

Epoch: 5
Iter: 800, Train Loss: 0.3031, Train Acc: 95.31%, Val Loss:  1.025, Val Acc: 85.54%, Time: 2.53s *
Iter: 900, Train Loss: 0.6932, Train Acc: 90.62%, Val Loss:  1.061, Val Acc: 86.68%, Time: 18.44s *
Epoch: 6
Iter: 1000, Train Loss: 0.3176, Train Acc: 95.31%, Val Loss:   1.09, Val Acc: 85.26%, Time: 2.92s 
Iter: 1100, Train Loss: 0.1158, Train Acc: 100.00%, Val Loss:  1.009, Val Acc: 87.54%, Time: 18.91s *
Epoch: 7
Iter: 1200, Train Loss: 0.6038, Train Acc: 95.31%, Val Loss: 0.9787, Val Acc: 88.03%, Time: 3.16s *
Iter: 1300, Train Loss: 0.09478, Train Acc: 100.00%, Val Loss: 0.9058, Val Acc: 88.11%, Time: 19.27s *
Epoch: 8
Iter: 1400, Train Loss: 0.2544, Train Acc: 96.88%, Val Loss:  1.006, Val Acc: 87.96%, Time: 3.43s 
Iter: 1500, Train Loss: 0.09064, Train Acc: 98.44%, Val Loss:   1.01, Val Acc: 87.18%, Time: 19.49s 
Epoch: 9
Iter: 1600, Train Loss: 0.06282, Train Acc: 100.00%, Val Loss:  0.987, Val Acc: 88.32%, Time: 3.76s *
Iter: 1700, Train Loss: 0.03998, Train Acc: 100.00

Iter: 1300, Train Loss: 0.1643, Train Acc: 95.31%, Val Loss: 0.8759, Val Acc: 88.25%, Time: 19.07s 
Epoch: 8
Iter: 1400, Train Loss: 0.06039, Train Acc: 100.00%, Val Loss: 0.8258, Val Acc: 89.17%, Time: 3.54s 
Iter: 1500, Train Loss: 0.07989, Train Acc: 100.00%, Val Loss: 0.7071, Val Acc: 90.88%, Time: 19.47s *
Epoch: 9
Iter: 1600, Train Loss: 0.09829, Train Acc: 98.44%, Val Loss: 0.8009, Val Acc: 88.96%, Time: 3.77s 
Iter: 1700, Train Loss: 0.07384, Train Acc: 98.44%, Val Loss:  0.832, Val Acc: 89.32%, Time: 19.69s 
Epoch: 10
Iter: 1800, Train Loss: 0.1256, Train Acc: 98.44%, Val Loss: 0.7834, Val Acc: 90.17%, Time: 4.01s 
Iter: 1900, Train Loss: 0.02012, Train Acc: 100.00%, Val Loss: 0.7055, Val Acc: 91.45%, Time: 19.84s *
Epoch: 11
Iter: 2000, Train Loss: 0.01693, Train Acc: 100.00%, Val Loss: 0.7961, Val Acc: 90.81%, Time: 4.34s 
Iter: 2100, Train Loss: 0.01219, Train Acc: 100.00%, Val Loss: 0.7675, Val Acc: 90.53%, Time: 20.18s 
Epoch: 12
Iter: 2200, Train Loss: 0.02657, Train Acc

Epoch: 9
Iter: 1600, Train Loss: 0.02884, Train Acc: 100.00%, Val Loss: 0.7646, Val Acc: 89.74%, Time: 3.75s 
Iter: 1700, Train Loss: 0.06581, Train Acc: 100.00%, Val Loss: 0.7813, Val Acc: 89.67%, Time: 19.67s 
Epoch: 10
Iter: 1800, Train Loss: 0.08723, Train Acc: 100.00%, Val Loss: 0.7578, Val Acc: 90.09%, Time: 3.99s 
1801:1400  No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.88      0.92      0.90       114
                   Filter       0.92      0.80      0.86       146
    Compute Derived Value       0.87      0.88      0.87       163
            Find Extremum       0.94      0.94      0.94       159
                     Sort       0.94      0.91      0.92       127
          Determine Range       0.83      0.92      0.87       143
Characterize Distribution       0.93      0.90      0.91       140
           Find Anomalies       0.84      0.9

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.953, Train Acc: 53.12%, Val Loss:   3.12, Val Acc: 48.33%, Time: 16.99s *
Epoch: 2
Iter: 200, Train Loss:  1.797, Train Acc: 73.44%, Val Loss:  1.909, Val Acc: 70.21%, Time: 1.71s *
Iter: 300, Train Loss:  1.261, Train Acc: 81.25%, Val Loss:  1.477, Val Acc: 77.55%, Time: 17.55s *
Epoch: 3
Iter: 400, Train Loss:  1.129, Train Acc: 81.25%, Val Loss:  1.156, Val Acc: 83.11%, Time: 1.97s *
Iter: 500, Train Loss: 0.8164, Train Acc: 85.94%, Val Loss:  1.055, Val Acc: 83.82%, Time: 17.79s *
Epoch: 4
Iter: 600, Train Loss: 0.7664, Train Acc: 87.50%, Val Loss: 0.9462, Val Acc: 85.53%, Time: 2.28s *
Iter: 700, Train Loss: 0.7214, Train Acc: 87.50%, Val Loss: 0.8696, Val Acc: 87.10%, Time: 18.22s *
Epoch: 5
Iter: 800, Train Loss:  0.312, Train Acc: 96.88%, Val Loss: 0.8128, Val Acc: 87.17%, Time: 2.59s *
Iter: 900, Train Loss: 0.2164, Train Acc: 95.31%, Val Loss: 0.8498, Val Acc: 87.38%, Time: 18.45s *
Epoch: 6
Iter: 1000, Train Loss:

Training and evaluating...
Epoch: 1
Iter: 100, Train Loss:  2.744, Train Acc: 59.38%, Val Loss:  3.534, Val Acc: 36.69%, Time: 16.59s *
Iter: 200, Train Loss:   2.03, Train Acc: 66.67%, Val Loss:  3.013, Val Acc: 52.12%, Time: 32.29s *
Epoch: 2
Iter: 300, Train Loss:  1.206, Train Acc: 81.25%, Val Loss:   2.75, Val Acc: 58.43%, Time: 15.72s *
Iter: 400, Train Loss:  1.335, Train Acc: 87.50%, Val Loss:  2.865, Val Acc: 57.55%, Time: 31.48s 
Epoch: 3
Iter: 500, Train Loss: 0.7322, Train Acc: 89.06%, Val Loss:  3.081, Val Acc: 56.75%, Time: 15.80s 
Iter: 600, Train Loss:  0.353, Train Acc: 95.83%, Val Loss:  3.086, Val Acc: 58.59%, Time: 31.68s *
Epoch: 4
Iter: 700, Train Loss:  0.215, Train Acc: 96.88%, Val Loss:  2.968, Val Acc: 60.91%, Time: 15.77s *
Iter: 800, Train Loss: 0.3956, Train Acc: 95.83%, Val Loss:  3.129, Val Acc: 60.51%, Time: 31.55s 
Epoch: 5
Iter: 900, Train Loss: 0.2294, Train Acc: 98.44%, Val Loss:  3.287, Val Acc: 60.67%, Time: 15.90s 
Iter: 1000, Train Loss: 0.4971, 

Iter: 700, Train Loss:  0.532, Train Acc: 93.75%, Val Loss:  2.206, Val Acc: 70.21%, Time: 19.74s 
Epoch: 5
Iter: 800, Train Loss: 0.2405, Train Acc: 98.44%, Val Loss:  2.397, Val Acc: 67.55%, Time: 4.50s 
Iter: 900, Train Loss: 0.2693, Train Acc: 93.75%, Val Loss:  2.325, Val Acc: 69.32%, Time: 20.62s 
Epoch: 6
Iter: 1000, Train Loss: 0.3223, Train Acc: 96.88%, Val Loss:  2.423, Val Acc: 71.47%, Time: 5.19s 
1001:600  No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
                           precision    recall  f1-score   support

           Retrieve Value       0.46      0.59      0.52       105
                   Filter       0.64      0.80      0.71       111
    Compute Derived Value       0.71      0.69      0.70       202
            Find Extremum       0.79      0.70      0.74       321
                     Sort       0.95      0.76      0.84       115
          Determine Range       0.47      0.52      0.49       125
Characterize Distributi

InvalidArgumentError: In[0] is not a matrix
	 [[Node: perturloss/Bi-LSTM/output/predictions/MatMul = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](perturloss/Bi-LSTM/Attention/dropout/mul, Bi-LSTM/outputW/read)]]
	 [[Node: add/_83 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_680_add", tensor_type=DT_DOUBLE, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'perturloss/Bi-LSTM/output/predictions/MatMul', defined at:
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-pa